In [163]:
import requests
import json
import os
from bs4 import BeautifulSoup
from datetime import datetime

In [190]:
source = 'activistpost'
date = '02-27'

filepath = '/mnt/c/Users/Brennan/Dropbox/MIDS/w210-capstone/data/notCredible/{0}/{1}.html'.format(source,date)

with open(filepath) as f:
    soup = BeautifulSoup(f, 'lxml')

In [191]:
# Pull out article links that were published on date
links = []

for div in soup.findAll('div', attrs={'class':'home-columns'}):
    for article in div.findAll('article'):
        for p in div.findAll('p', attrs={'class':'entry-meta'}):
            for span in p.findAll('span', attrs={'class':'entry-meta-date updated'}):
                pubdate = datetime.strptime(span.text, "%B %d, %Y")
                if date in str(pubdate):
                    for header in article.findAll('h3'):
                        for link in header.findAll('a'):
                            links.append(link.get('href'))
                    
links = set(links)
links = list(links)

In [192]:
i=0
for link in links:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')
    title = soup.title.string
    date_longform = '{0}-2017'.format(date)
    contents = []
    for div in soup.findAll('div', attrs={'class': 'entry-content clearfix'}):
        for p in div.findAll('p'):
            contents.append(p)
    article = {}
    content = ''.join(str(p) for p in contents)
    article["title"] = title
    article["url"] = link
    article["date"] = date_longform
    article["source"] = source
    article["content"] = content
    
    outfile = '/mnt/c/Users/Brennan/Dropbox/MIDS/w210-capstone/data/notCredible/{0}/articles/{2}/article{1}_{2}.txt'.format(source,i,date)
    os.makedirs(os.path.dirname(outfile), exist_ok=True)
    i+=1

    with open(outfile, 'w') as f:
        json.dump(article, f)